<h2>3D stack - Batch Processing - Marker+ based on colocalization</h2>

The following notebook is able to process a 3D stack (.czi or .nd2 files) into a MIP and allows you to:

1. Read previously defined ROIs, if not present, full image is analyzed.
2. Read previously predicted nuclei labels, if not present, generates them.
3. Extract numbers of cells positive for a marker based on colocalization (using a user-defined threshold).
4. Save number and % of positive cells in a .csv file (BP_marker_+_label_coloc).

In [3]:
from pathlib import Path
import tifffile
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
from utils_stardist import get_gpu_details, list_images, read_image, extract_nuclei_stack, get_stardist_model, simulate_cytoplasm_chunked_3d, simulate_cell_chunked_3d, simulate_cytoplasm, simulate_cell, segment_marker_positive_labels, segment_nuclei, remove_labels_touching_roi_edge

get_gpu_details()
cle.select_device("RTX")

Device name: /device:GPU:0
Device type: GPU
GPU model: device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (1 refs)>

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [4]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# cellular locations can be "nucleus", "cytoplasm" or "cell" (cell being the sum volume of nucleus and cytoplasm)
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]
markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

# Iterate through the .czi and .nd2 files in the directory
images = list_images(directory_path)

images

['..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI1_CONTRA_M8_S7_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR1.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S6_TR2.czi',
 '..\\raw_data\\test_data\\HI1_IPSI_M8_S7_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_CONTRA_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI2_IPSI_M10_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_CONTRA_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR1.czi',
 '..\\raw_data\\test_data\\HI3_IPSI_M11_S10_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR1.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M6_S11_TR2.czi',
 '..\\raw_data\\test_data\\SHAM1_CONTRA_M7_S11_TR1.czi',
 '..\\raw_data\\t

<h3>Define your batch analysis parameters</h3>

If you have generated nuclei predictions already, make sure to input the same <code>slicing factor</code> you used when generating nuclei predictions. 

If you have not generated nuclei predictions before, input <code>nuclei_channel</code>, <code>n_tiles</code>, <code>segmentation_type</code> and <code>model_name</code> values.

In [5]:
# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = None # Use 2 to select 1 out of every 2 z-slices

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# The n_tiles parameter defines the number of tiles the input volume/image will be divided into along each dimension (z, y, x) during prediction. 
# This is useful for processing large images that may not fit into memory at once.
# While tiling can handle memory limitations, chopping the image into smaller chunks increases
# the processing time for stitching the predictions back together. 
# Use n_tiles=(1, 1, 1) if the input volume fits in memory without tiling to minimize processing overhead.
n_tiles=(1,4,4)

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "3D"

# Nuclear segmentation model type ("Stardist")
# Choose your Stardist fine-tuned model (model_name) from stardist_models folder
# If no custom model is present, type "test" and a standard pre-trained model will be loaded
model_name = "MEC0.1" # Type "test" if you don't have a custom model trained

# Model loading 
model = get_stardist_model(segmentation_type, name=model_name, basedir='stardist_models')

Loading 3D segmentation model
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.583933, nms_thresh=0.3.


Define in <code>parameters_per_marker</code> the <code>marker</code> you want to use to define your cell populations of interest, the <code>min_max</code> range of pixel intensity values and the <code>population</code> name.

In addition, set the <code>erosion_factor</code> and <code>cytoplasm_dilation_radius</code> for each marker you want to analyze.

In [ ]:
# min_max range defines the pixel intensity range within which a cell is considered positive for a marker

# erosion_factor sets the amount of erosion that is applied to areas where the marker+ signal colocalizes with nuclear or cytoplasmic signal
# The higher the value, the stricter the conditions to consider a nuclei as marker+

# cytoplasm_dilation_radius sets the amount of pixels you want to add around the nuclei to simulate the cytoplasm

parameters_per_marker = [{"marker": "ki67", "min_max_range": (200, 255), "population": "ki67", "erosion_factor":4, "cytoplasm_dilation_radius":0},
                      {"marker": "neun", "min_max_range": (50, 115), "population": "neun_low", "erosion_factor":4, "cytoplasm_dilation_radius":2},
                      {"marker": "neun", "min_max_range": (115, 255), "population": "neun_high", "erosion_factor":4, "cytoplasm_dilation_radius":2},
                      {"marker": "calbindin", "min_max_range": (65, 255),"population": "calbindin", "erosion_factor":4,  "cytoplasm_dilation_radius":2}]

<h3>Run Batch Analysis</h3>

In [ ]:
# Construct ROI and nuclei predictions paths from directory_path above
roi_path = directory_path / "ROIs"
nuclei_preds_path =  directory_path / "nuclei_preds" / segmentation_type / model_name

# Extract the experiment name from the data directory path
experiment_id = directory_path.name

# Define output folder for results
results_folder = Path("results") / experiment_id / segmentation_type / model_name

# Create the necessary folder structure if it does not exist
try:
    os.makedirs(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")

# List of subfolder names
try:
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]

except FileNotFoundError:
    roi_names = ["full_image"]
        
print(f"The following regions of interest will be analyzed: {roi_names}")

for image in tqdm(images):

    # Create an empty list to store all stats extracted from each image
    stats = []

    # Initialize a combined DataFrame with the expected columns for the current image 
    columns = ["filename", "ROI", "label"] + [param["population"] for param in parameters_per_marker]
    combined_df = pd.DataFrame(columns=columns)

    # Read image, apply slicing if needed and return filename and img as a np array
    img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

    for roi_name in roi_names:

        print(f"\nAnalyzing ROI: {roi_name}")

        # Read the user defined ROIs, in case of full image analysis generate a label covering the entire image
        try:
            # Read previously defined ROIs
            user_roi = tifffile.imread(roi_path / roi_name / f"{filename}.tiff")

        except FileNotFoundError:
            # Extract the xy dimensions of the input image 
            img_shape = img.shape
            img_xy_dims = img.shape[-2:]

            # Create a label covering the entire image
            user_roi = np.ones(img_xy_dims).astype(np.uint8)

        # Read previously predicted nuclei labels, if not present generate nuclei predictions and save them
        try:
            # Read the nuclei predictions per ROI
            nuclei_labels = tifffile.imread(nuclei_preds_path / roi_name / f"{filename}.tiff")
            print(f"Pre-computed nuclei labels found for {filename}")
            # Remove labels touching ROI edge (in place for nuclei predictions generated before "remove_labels_touching_roi_edge" was implemented)
            print("Removing nuclei labels touching ROI edge")
            nuclei_labels = remove_labels_touching_roi_edge(nuclei_labels, user_roi)

        except FileNotFoundError:
            print(f"Generating nuclei labels for {filename}")

            # If 3D-segmentation input nuclei_img is a 3D-stack
            if segmentation_type == "3D":
                # Slice the nuclei stack
                nuclei_img = extract_nuclei_stack(img, nuclei_channel)

            # If 2D-segmentation input nuclei_img is a max intensity projection of said 3D-stack
            elif segmentation_type == "2D":
                # Slice the nuclei stack
                nuclei_img = extract_nuclei_stack(img, nuclei_channel)
                nuclei_img = np.max(nuclei_img, axis=0)

            # We will create a mask where roi is greater than or equal to 1
            mask = (user_roi >= 1).astype(np.uint8)

            # 3D segmentation logic, extend 2D mask across the entire stack volume
            if segmentation_type == "3D":
                # Extract the number of z-slices to extend the mask
                slice_nr = img.shape[1]
                # Extend the mask across the entire volume
                mask = np.tile(mask, (slice_nr, 1, 1))
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)
            else:
                # Apply the mask to nuclei_img, setting all other pixels to 0
                masked_nuclei_img = np.where(mask, nuclei_img, 0)

            # Segment nuclei and return labels
            nuclei_labels = segment_nuclei(masked_nuclei_img, segmentation_type, model, n_tiles)

            # Remove labels touching ROI edge
            print("Removing nuclei labels touching ROI edge")
            nuclei_labels = remove_labels_touching_roi_edge(nuclei_labels, user_roi)

            # Save nuclei labels as .tiff files to reuse them later
            try:
                os.makedirs(nuclei_preds_path / roi_name, exist_ok=True)
            except Exception as e:
                print(f"Error creating directory {nuclei_preds_path / roi_name}: {e}")

            # Construct path to store
            path_to_store = nuclei_preds_path / roi_name / f"{filename}.tiff"
            print(f"Saving nuclei labels to {path_to_store}")
            try:
                tifffile.imwrite(path_to_store, nuclei_labels)
            except Exception as e:
                print(f"Error saving file {path_to_store}: {e}")

        for marker_parameters in parameters_per_marker:

            # Extract info from list of dictionaries and open marker_img
            marker_name = marker_parameters["marker"]
            min_max_range = marker_parameters["min_max_range"]
            population = marker_parameters["population"]
            erosion_factor = marker_parameters["erosion_factor"]
            cytoplasm_dilation_radius = marker_parameters["cytoplasm_dilation_radius"]

            print(f"Analyzing marker/population: {marker_name}/{population}")

            # Retrieve the first and second values (channel and location) of the corresponding tuple in markers
            for item in markers:
                if item[0] == marker_name:
                    marker_channel = item[1]
                    location = item[2]
                    break  # Stop searching once the marker is found

            # Access the corresponding marker intensity image
            marker_img = img[marker_channel, :, :, :]

            # Select marker positive cell compartments
            if location == "nucleus":
                # Select marker positive nuclei
                nuclei_and_marker, eroded_nuclei_and_marker, marker, processed_region_labels = segment_marker_positive_labels(nuclei_labels, marker_img, min_max_range, erosion_factor, segmentation_type)

            # Simulate a cytoplasm by growing the nuclei_labels and masking out the inner nucleus
            if location == "cytoplasm":
                if segmentation_type == "3D":
                    print(f"Generating {segmentation_type} cytoplasm labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    cytoplasm_labels = simulate_cytoplasm_chunked_3d(nuclei_labels, dilation_radius=cytoplasm_dilation_radius, erosion_radius=0, chunk_size=(1, 1024, 1024))

                elif segmentation_type == "2D":
                    print(f"Generating {segmentation_type} cytoplasm labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    cytoplasm_labels = simulate_cytoplasm(nuclei_labels, dilation_radius=cytoplasm_dilation_radius, erosion_radius=0)
                
                # Select marker positive cytoplasms
                cytoplasm_and_marker, eroded_cytoplasm_and_marker, marker, processed_region_labels = segment_marker_positive_labels(cytoplasm_labels, marker_img, min_max_range, erosion_factor, segmentation_type)

            # Simulate a cell by growing the nuclei_labels
            elif location == "cell":
                if segmentation_type == "3D":
                    print(f"Generating {segmentation_type} cell labels for: {marker_name}")
                    # Simulate a cell volume by dilating the nuclei 
                    cell_labels = simulate_cell_chunked_3d(nuclei_labels, dilation_radius=cytoplasm_dilation_radius, erosion_radius=0, chunk_size=(1, 1024, 1024))

                elif segmentation_type == "2D":
                    print(f"Generating {segmentation_type} cell labels for: {marker_name}")
                    # Simulate a cytoplasm by dilating the nuclei and subtracting the nuclei mask afterwards
                    cell_labels = simulate_cell(nuclei_labels, dilation_radius=cytoplasm_dilation_radius, erosion_radius=0)

                # Select marker positive cells
                cell_and_marker, eroded_cell_and_marker, marker, processed_region_labels = segment_marker_positive_labels(cell_labels, marker_img, min_max_range, erosion_factor, segmentation_type)

            # Store per label results in a dataframe

            # Get unique positive labels
            #Finds unique values in an array and removes background label (0)
            positive_labels = np.unique(processed_region_labels[processed_region_labels != 0])

            # Generate the label column with all labels
            max_label = nuclei_labels.max()
            label_column = np.arange(1, max_label + 1)

            # Check if positive_labels is in label_column and set values to True 
            channel_column = np.isin(label_column, positive_labels)

            # Create the DataFrame to hold per label data
            df_temp = pd.DataFrame({
                "filename": [filename] * len(label_column),
                "ROI": [roi_name] * len(label_column),
                'label': label_column,
                population: channel_column
            })

            # Ensure population columns exist in combined_df
            for col in [population]:
                if col not in combined_df.columns:
                    combined_df[col] = False

            # Handle potential duplicate columns during merge
            combined_df = combined_df.merge(df_temp, on=["filename", "ROI", "label"], how="outer", suffixes=('', '_dup'))

            # Drop duplicate columns while retaining original values from df_temp
            for col in combined_df.columns:
                if col.endswith('_dup'):
                    # Use .where() to retain the original values
                    combined_df[col[:-4]] = combined_df[col[:-4]].where(combined_df[col[:-4]].notna(), combined_df[col])

                    # Explicitly cast the column to the correct type (e.g., bool or int)
                    combined_df[col[:-4]] = combined_df[col[:-4]].astype(bool)  # Change to int if needed
                    
                    # Drop the duplicate column
                    combined_df.drop(columns=[col], inplace=True)

            # Define the .csv path
            per_filename_csv_path = results_folder / f"{filename}_per_label_coloc.csv"

            # Save per label data on a per filename basis
            combined_df.to_csv(per_filename_csv_path)

            # Extract your information of interest
            total_nuclei = len(np.unique(nuclei_labels)) - 1
            marker_pos_nuclei = len(np.unique(processed_region_labels)) - 1

            # Calculate "%_marker+_cells" and avoid division by zero errors
            try:
                perc_marker_pos_cells = (marker_pos_nuclei * 100) / total_nuclei
            except ZeroDivisionError:
                perc_marker_pos_cells = 0

            # Create a dictionary containing all extracted info per masked image
            stats_dict = {
                        "filename": filename,
                        "ROI": roi_name,
                        "population": population,
                        "marker": marker_name,
                        "marker_location":location,
                        "total_nuclei": total_nuclei,
                        "marker+_nuclei": marker_pos_nuclei,
                        "%_marker+_cells": perc_marker_pos_cells,
                        "nuclei_ch": nuclei_channel,
                        "marker_ch": marker_channel,
                        "min_max_avg_int": min_max_range,
                        "cytoplasm_dilation":cytoplasm_dilation_radius,
                        "erosion_factor": erosion_factor,
                        "slicing_factor_xy": slicing_factor_xy,
                        "slicing_factor_z": slicing_factor_z
                        }

            # Append the current data point to the stats_list
            stats.append(stats_dict)

    # Transform into a dataframe to store it as .csv later
    df = pd.DataFrame(stats)

    # Define the .csv path
    csv_path = results_folder / f"BP_marker_+_label_coloc.csv"

    # Append to the .csv with new data points each round
    df.to_csv(csv_path, mode="a", index=False, header=not os.path.isfile(csv_path))

# Show the updated .csv 
csv_df = pd.read_csv(csv_path)

csv_df    

Output folder created: results\test_data\3D\MEC0.1
The following regions of interest will be analyzed: ['CA', 'DG']


  0%|          | 0/23 [00:00<?, ?it/s]



Image analyzed: HI1_CONTRA_M8_S6_TR1
Original Array shape: (4, 14, 3803, 2891)
Compressed Array shape: (4, 14, 3803, 2891)

Analyzing ROI: CA
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Removing nuclei labels touching ROI edge
Analyzing marker/population: ki67/ki67
Analyzing marker/population: neun/neun_low
Generating 3D cell labels for: neun
Analyzing marker/population: neun/neun_high
Generating 3D cell labels for: neun
Analyzing marker/population: calbindin/calbindin
Generating 3D cytoplasm labels for: calbindin


utils_stardist.py (520): Mismatch in label sets! Nuclei labels: 1510, Cytoplasm labels: 1508



Analyzing ROI: DG
Pre-computed nuclei labels found for HI1_CONTRA_M8_S6_TR1
Removing nuclei labels touching ROI edge
Analyzing marker/population: ki67/ki67
Analyzing marker/population: neun/neun_low
Generating 3D cell labels for: neun
Analyzing marker/population: neun/neun_high
Generating 3D cell labels for: neun
Analyzing marker/population: calbindin/calbindin
Generating 3D cytoplasm labels for: calbindin


...



Image analyzed: SHAM1_IPSI_M7_S11_TR2
Original Array shape: (4, 15, 2883, 2889)
Compressed Array shape: (4, 15, 2883, 2889)

Analyzing ROI: CA
Pre-computed nuclei labels found for SHAM1_IPSI_M7_S11_TR2
Removing nuclei labels touching ROI edge
Analyzing marker/population: ki67/ki67
Analyzing marker/population: neun/neun_low
Generating 3D cell labels for: neun
Analyzing marker/population: neun/neun_high
Generating 3D cell labels for: neun
Analyzing marker/population: calbindin/calbindin
Generating 3D cytoplasm labels for: calbindin


utils_stardist.py (520): Mismatch in label sets! Nuclei labels: 807, Cytoplasm labels: 806



Analyzing ROI: DG
Pre-computed nuclei labels found for SHAM1_IPSI_M7_S11_TR2
Removing nuclei labels touching ROI edge
Analyzing marker/population: ki67/ki67
Analyzing marker/population: neun/neun_low
Generating 3D cell labels for: neun
Analyzing marker/population: neun/neun_high
Generating 3D cell labels for: neun
Analyzing marker/population: calbindin/calbindin
Generating 3D cytoplasm labels for: calbindin


utils_stardist.py (520): Mismatch in label sets! Nuclei labels: 1332, Cytoplasm labels: 1326
100%|██████████| 23/23 [35:51<00:00, 93.52s/it] 


,filename,ROI,population,marker,marker_location,total_nuclei,marker+_nuclei,%_marker+_cells,nuclei_ch,marker_ch,min_max_avg_int,cytoplasm_dilation,erosion_factor,slicing_factor_xy,slicing_factor_z
0,HI1_CONTRA_M8_S6_TR1,CA,ki67,ki67,nucleus,1509,8,0.530152,3,0,"(200, 255)",0,4,NaN,NaN
1,HI1_CONTRA_M8_S6_TR1,CA,neun_low,neun,cell,1509,4,0.265076,3,1,"(50, 115)",0,4,NaN,NaN
2,HI1_CONTRA_M8_S6_TR1,CA,neun_high,neun,cell,1509,305,20.212061,3,1,"(115, 255)",0,4,NaN,NaN
3,HI1_CONTRA_M8_S6_TR1,CA,calbindin,calbindin,cytoplasm,1509,0,0.000000,3,2,"(65, 255)",2,4,NaN,NaN
4,HI1_CONTRA_M8_S6_TR1,DG,ki67,ki67,nucleus,2164,20,0.924214,3,0,"(200, 255)",0,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,SHAM1_IPSI_M7_S11_TR2,CA,calbindin,calbindin,cytoplasm,806,0,0.000000,3,2,"(65, 255)",2,4,NaN,NaN
180,SHAM1_IPSI_M7_S11_TR2,DG,ki67,ki67,nucleus,1331,0,0.000000,3,0,"(200, 255)",0,4,NaN,NaN
181,SHAM1_IPSI_M7_S11_TR2,DG,neun_low,neun,cell,1331,2,0.150263,3,1,"(50, 115)",0,4,NaN,NaN
182,SHAM1_IPSI_M7_S11_TR2,DG,neun_high,neun,cell,1331,251,18.858002,3,1,"(115, 255)",0,4,NaN,NaN
